<a href="https://colab.research.google.com/github/ramscodehub/Auto-Tag-Images-of-Gala/blob/master/Copy_of_Copy_of_autotagImagesGalaFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required libraries
#try:
  #%tensorflow_version 2.x
#except Exception:
  #pass
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras.preprocessing import image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
#print(tf.__version__)

Using TensorFlow backend.


In [ ]:
#mounting google drive
from google.colab import drive
drive.mount("/content/drive")
#getting data directly from drive to colab for loading images fastly
!cp -r /content/drive/My\ Drive/Galas/dataset/trainImagesGala /content/
!cp -r /content/drive/My\ Drive/Galas/dataset/testImagesGala /content/
!cp -r /content/drive/My\ Drive/Galas/dataset/trainGala.csv /content/
!cp -r /content/drive/My\ Drive/Galas/dataset/testGala.csv /content/
train_data=pd.read_csv("/content/trainGala.csv")
test_data=pd.read_csv("/content/testGala.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#checking the dimensionalities of training and testing samples
print("train_data",end=" ")
print(train_data.shape)
print("test_data",end=" ")
print(test_data.shape)
train_data.head()

train_data (5983, 2)
test_data (3219, 1)


,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [ ]:
#converting categorical values i.e., classes  to numerical values for training using dictionaries dataStructure
Class_map={"Food":0,"Attire":1,"Decorationandsignage":2,"misc":3}
inverse_map={0:"Food",1:"Attire",2:"Decorationandsignage",3:"misc"}
train_data["Class"]=train_data["Class"].map(Class_map)
train_data.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [ ]:
#Data Precprocessing and loading  of Training Samples 
# importing matplotlib modules 
train_images=[]
train_labels=[]
j=0
initial_path="/content/trainImagesGala"
for i in tqdm(train_data["Image"]):
  final_path=os.path.join(initial_path,i)
  #print(final_path)
  pic = image.load_img(final_path,target_size=(224, 224))
  #imag=cv2.resize(imag,(224,224))
  #imag=imag.astype("float32")
  imag= image.img_to_array(pic)
  imag=np.expand_dims(imag,axis=0)
  imag=preprocess_input(imag)
  train_images.append(imag)
  train_labels.append(train_data["Class"][j])
  j=j+1

In [ ]:
#Preprocessing and loading of Test Samples
test_images=[]
initial_path2="/content/testImagesGala"
for i in tqdm(test_data["Image"]):
  final_path2=os.path.join(initial_path2,i)
  pic=image.load_img(final_path2,target_size=(224,224))
  imag=image.img_to_array(pic)
  imag=np.expand_dims(imag,axis=0)
  imag=preprocess_input(imag)
  test_images.append(imag)

In [ ]:
#Images
from keras.utils import np_utils
train_images = np.array(train_images)
print(train_images.shape,"1st")
train_images = np.rollaxis(train_images, 1, 0)
print(train_images.shape,  "2nd")
X_train = train_images[0]
print(X_train.shape,   "3rd")
#Labels
train_labels = np.array(train_labels)
Y_train = np_utils.to_categorical(train_labels,4)
print(Y_train.shape,   "4th")

(5983, 1, 224, 224, 3) 1st
(1, 5983, 224, 224, 3) 2nd
(5983, 224, 224, 3) 3rd
(5983, 4) 4th


In [ ]:
test_images=np.array(test_images)
print(test_images.shape,  "1st")
test_images=np.rollaxis(test_images,1,0)
print(test_images.shape,  "2nd")
X_test=test_images[0]
print(X_test.shape,  "3rd")

(3219, 1, 224, 224, 3) 1st
(1, 3219, 224, 224, 3) 2nd
(3219, 224, 224, 3) 3rd


In [ ]:
#Building the actual model using Transfer Learning
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications.mobilenet import MobileNet
IMAGE_SIZE=[224,224]
sample_model=MobileNet(include_top=False,weights='imagenet',input_shape=[*IMAGE_SIZE,3])
sample_model.trainable=False
actual_model=Sequential([sample_model,
                         tf.keras.layers.GlobalAveragePooling2D(),
                         Dense(256,activation='relu'),
                         Dense(4,activation='softmax')])

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)
callbacks=[reduce_learning_rate]


#Compiling the actual_model
actual_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
actual_model.fit(X_train,Y_train,batch_size=32,epochs=10,callbacks=callbacks)

Epoch 1/10
187/187 [==============================] - 24s 127ms/step - loss: 1.0660 - accuracy: 0.5674 - lr: 0.0010
Epoch 2/10
187/187 [==============================] - 23s 125ms/step - loss: 0.8553 - accuracy: 0.6522 - lr: 0.0010
Epoch 3/10
187/187 [==============================] - 23s 125ms/step - loss: 0.7775 - accuracy: 0.6838 - lr: 0.0010
Epoch 4/10
187/187 [==============================] - 23s 124ms/step - loss: 0.7137 - accuracy: 0.7165 - lr: 0.0010
Epoch 5/10
187/187 [==============================] - 23s 124ms/step - loss: 0.6605 - accuracy: 0.7369 - lr: 0.0010
Epoch 6/10
187/187 [==============================] - 23s 124ms/step - loss: 0.5844 - accuracy: 0.7729 - lr: 0.0010
Epoch 7/10
187/187 [==============================] - 23s 123ms/step - loss: 0.5179 - accuracy: 0.7998 - lr: 0.0010
Epoch 8/10
187/187 [==============================] - 23s 123ms/step - loss: 0.4516 - accuracy: 0.8277 - lr: 0.0010
Epoch 9/10
187/187 [==============================] - 23s 123ms/step - l

In [ ]:
#Using the trained model for predictions
test_labels=actual_model.predict(X_test)
testClassLabels=[]
for i in test_labels:
  testClassLabels.append(np.argmax(i))
class_label=[inverse_map[x] for x in testClassLabels]

In [ ]:
submission=pd.DataFrame({"Image":test_data.Image,"Class":class_label})
submission.to_csv('/content/drive/My Drive/oooops.csv', index=False)